In [ ]:
# Loader
from langchain_community.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader

# Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# Store
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Chroma

# Tokenization
from transformers import AutoTokenizer

# Language model
from transformers import RobertaForSequenceClassification

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, Dataset, TensorDataset
from torch.nn.functional import softmax

# Additional libraries
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import time
import numpy as np
from collections import Counter
import itertools
import matplotlib.pyplot as plt

#### **Load and split new document**

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap = 100, add_start_index = False) # splits the text into chunks
loader = PyPDFLoader("/home/user123/contradictory-information/data/Inject/Press_release_q1_2024.pdf").load()
split_text = text_splitter.split_documents(loader)

In [ ]:
inject_content = [doc.page_content for doc in split_text]
print(f"Number of chunks in new document: {len(inject_content)}")

#### **Similarity search**

In [ ]:
# Load embedding and vector store
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L12-v2")
db = Chroma(persist_directory="/home/user123/contradictory-information/src/vectorization/vector_store/kb_langchain", embedding_function=embedding_function)

In [ ]:
knowledge_chunks = []
k = 2
for chunk in inject_content:
    docs = db.similarity_search_with_score(chunk, k=k)
    #print(docs[0][1])
    #if docs[0][1] <= 0.5:
    #    print(docs[0][0])

    for doc, score in docs:
        knowledge_chunks.append(doc.page_content)

(f"Number of similar chunks in knowledge base: {len(knowledge_chunks)}")

#### **Create and clean pairs**

In [ ]:
# Cleaning each premise
cleaned_premises = [
    re.sub(r'\s+\.', '.',  # Remove space before period
           re.sub(r'\s+,', ',',  # Remove space before comma
                  re.sub(r'\n', '', chunk)))  # Remove new lines
    for chunk in inject_content]

sentences_premises = [sent_tokenize(text) for text in cleaned_premises]
len(sentences_premises)

In [ ]:
# Cleaning each hypothesis
cleaned_hypotheses = [
    re.sub(r'\s+\.', '.',  # Remove space before period
           re.sub(r'\s+,', ',',  # Remove space before comma
                  re.sub(r'\n', '', chunk)))  # Remove new lines
    for chunk in knowledge_chunks]

sentences_hypotheses = [sent_tokenize(text) for text in cleaned_hypotheses]
len(sentences_hypotheses)

In [ ]:
# Generate combination of each sentence on one chunk for each sentence in the corresponding chunk
#similar_pairs = [list(itertools.product(sentences_premises[k], sentences_hypotheses[k])) for k in range(0, len(sentences_premises))]
#similar_pairs = list(itertools.chain.from_iterable(similar_pairs)) # de-nest list of lists

#print(f"Number of sentence pairs: {len(similar_pairs)}")

In [ ]:
# Generate combination of each sentence on one chunk for each sentence in the corresponding chunk
similar_pairs = []
num_pairs = k
# Loop through the sentence_premises
for i in range(len(sentences_premises)):
    # Create pairs for the specified number of corresponding strings from sentence_hypothesis
    for j in range(num_pairs):
        # Check to prevent index out of range error
        if num_pairs * i + j < len(sentences_hypotheses):
            pair = list(itertools.product(sentences_premises[i], sentences_hypotheses[num_pairs*i + j]))
            similar_pairs.append(pair)

similar_pairs = list(itertools.chain.from_iterable(similar_pairs)) # de-nest list of lists
print(f"Number of sentence pairs: {len(similar_pairs)}")

#### **Classification of pairs**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device type:', device)
print('There are %d GPU(s) available.' % torch.cuda.device_count())
print('GPU is:', torch.cuda.get_device_name(0))

In [ ]:
model_path = '/home/user123/contradictory-information/model/roberta-base.pt'
#model_path = '/home/user123/contradictory-information/model/xlm-roberta-base-snli-mnli-anli-xnli.pt'
#model_path = '/home/user123/contradictory-information/model/roberta-large.pt'

tokenizer_path = 'roberta-base'
#tokenizer_path = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'
#tokenizer_path = 'roberta-large'

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = torch.load(model_path)
model.to(device)

MAX_LENGTH = 256

# Function for encoding input data
def encode_sets(data, tokenizer):
    kwargs = { 'truncation': True,
    'max_length': MAX_LENGTH,
    'padding': 'max_length',
    'return_attention_mask': True, 
    'return_token_type_ids': True     
    }
    #datalist = list(zip(data['premise'], data['hypothesis']))
    tokenized = tokenizer.batch_encode_plus(data,**kwargs)
    input_ids = torch.LongTensor(tokenized.input_ids)
    attention_masks = torch.LongTensor(tokenized.attention_mask)
    token_type_ids = torch.LongTensor(tokenized.token_type_ids)
    return input_ids, attention_masks, token_type_ids


# Encode similar pairs
input_ids, attention_masks, token_type_ids = encode_sets(similar_pairs, tokenizer)
pairs_tensor = TensorDataset(input_ids, attention_masks, token_type_ids)
pairs_dataloader = DataLoader(pairs_tensor, sampler=SequentialSampler(pairs_tensor), batch_size=len(similar_pairs))

model.eval()

for batch in pairs_dataloader:

    with torch.no_grad():
        input_ids, attention_masks, token_type_ids = (batch[0].to(device), 
                                                      batch[1].to(device),
                                                      batch[2].to(device))
        start_time = time.time()
        outputs = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_masks)
        end_time = time.time()

    logits = outputs.logits
    model_preds = logits.detach().cpu().numpy()

    # Calculate inference time
    inference_time = end_time - start_time
    print(f"\nInference time: {inference_time:.4f} seconds\n")
    predictions = np.argmax(model_preds, axis=1)

# target names according to 3 label or 11 label model
if model_path == '/home/user123/contradictory-information/model/xlm-roberta-base-snli-mnli-anli-xnli.pt':
    target_names = [
    "entailment",
    "neutral",
    "contradiction"
]
else:
    target_names = [
    "antonymity",
    "entailment",
    "factive_antonymity",
    "factive_embedding_verb",
    "lexical",
    "negation",
    "neutral",
    "numeric",
    "structure",
    "temporal",
    "worldknowledge"
]
    
# Use Counter to count occurrences
count = {target_names[key]: value for key, value in Counter(predictions).items()}

# Printing the count of each number
print(f"Predictions:\n{count}")

# count all predicted labels
total_count = sum(count.values())

# check whether label entailment has been predicted at least once and count the amount
if 'entailment' in count.keys():
    entailment_share = (count["entailment"] / total_count) * 100
else:
    entailment_share = 0

# check whether label neutral has been predicted at least once and count the amount
if 'neutral' in count.keys():
    neutral_share = (count["neutral"] / total_count) * 100
else:
    neutral_share = 0

# check whether a contradiction label has been predicted at least once count the amount
keys = {'entailment', 'neutral'}
contradiction_keys = set(count.keys()) - keys
if contradiction_keys:
    contradiction_labels =  [label for label in count if label not in ["entailment", "neutral"]]
    contradiction_count = sum(count[label] for label in contradiction_labels)
    contradiction_share = (contradiction_count / total_count) * 100
else:
    contradiction_share = 0

print(f"\nDistribution:\nEntailment: {entailment_share:.2f}%, Neutral: {neutral_share:.2f}%, Contradictions: {contradiction_share:.2f}%")

In [ ]:
# calculate probabilty for each predicted class
probabilities = softmax(logits, dim=1)
#probabilities = [[round(max(value), 5), target_names[value.index(max(value))]] for value in probabilities.tolist()]
probabilities = [round(max(value), 2) for value in probabilities.tolist()]

# join pairs with label and probability
labels = [target_names[number] for number in predictions]
classifications = list(zip(similar_pairs, labels))
list(zip(classifications, probabilities))
contradictions = [[[probability, label], premise, hypothesis] for (((premise, hypothesis), label), probability) in list(zip(classifications, probabilities)) if label != 'entailment' and label != 'neutral']
classifications = [[label, premise, hypothesis] for (((premise, hypothesis), label), probability) in list(zip(classifications, probabilities))]
for number, pair in enumerate(classifications):
    print(number, pair)

In [ ]:
manual_contradictions = [['Group revenue in the first quarter of 2024 grew by 2.8 percent to € 2.859 billion yearon-year (Q1 2023: € 2.781 billion).','Group revenue in the first quarter of 2024 grew by 5.4 percent to € 2.859 billion yearon-year (Q1 2023: € 2.781 billion).', 'numeric'],
                         ['In the Industrial Trucks & Services segment, revenue increased by 7.4 percent to € 2.153 billion (Q1 2023: € 2.005 billion), mainly due to the positive geographic and product mix as well as higher production output and sales prices. The service business also grew.', 'In the Industrial Trucks & Services segment, revenue decreased by 7.4 percent to € 2.153 billion (Q1 2023: € 2.005 billion), mainly due to the negative geographic and product mix as well as higher production output and sales prices.', 'antonymity'],
                         ['This was mainly due to the continued stability of material purchase prices, increased productivity as a result of improved material availability and revenue growth.', 'This was to a small degree due to the continued stability of material purchase prices, increased productivity as a result of improved material availability and revenue growth.', 'factive_antonymity'],
                         ['The KION Group is one of the world’s leading providers of industrial trucks and supply chain solutions.','The KION Group has removed industrial trucks from its product portfolio and focuses on supply chain solutions.', 'worldknowledge'],
                         ['Based on revenue for the year 2022, the KION Group is the leading overseas manufacturer in China, and including domestic manufacturers, the third-largest supplier there.', 'Based on revenue for the year 2022, the KION Group is the leading overseas manufacturer in China, and including domestic manufacturers, the second-largest supplier there.','structure'],
                         ['The KION Group is also one of the world’s leading warehouse automation providers, based on 2022 revenue.','The KION Group has lost its leading position as a warehouse automation provider, based on 2022 revenue.','factive_embedding_verb'],
                         ['At the end of 2023, more than 1.8 million industrial trucks of the KION Group were in use by customers from all manner of sectors and of varying sizes on six continents.','At the end of 2023, more than 1.8 million industrial trucks of the KION Group were in use by customers from a small number of sectors and of same sizes on six continents.','lexical'],
                         ]

reverse = [[hypothesis] for [premise, hypothesis, label] in manual_contradictions]

In [ ]:
reverse

In [ ]:
my_contradictions = [['temporal', 'EBIT  margin of  7.9 percent  (Q1 2023: 5.6 percent)  \uf0a7 Positive  free cash flow of € 65.7 million  (Q1 2023: € 104.9 million ) \uf0a7 Full year 2024 outlook confirmed   Frankfurt am Main, April 25th,  2024  – The KION Group started the financial year 2024 with revenue  growth and improved  profitability.', 'Frankfurt am Main, April 25th, 2024 – The KION Group just finished  the financial year 2024 and started 2025 with revenue growth and improved profitability.'],
    
['numeric', 'Group revenue in the first quarter of 2024 grew by 2.8 percent to € 2.859 billion year -on-year (Q1 2023: € 2.781 billion).', 'Group revenue in the first quarter of 2024 grew by 5.4 percent  to € 2.859 billion year - on-year ( Q1 2023: € 2.781 billion).'],

['structure', 'In the Industrial Trucks & Services  segment, revenue increased by 7.4 percent  to € 2.153 billion ( Q1 2023: € 2.005 billion), mainly due to the positive geographic and product mix as well as  higher  production output  and', 'In the Industrial Trucks & Services segment,  revenue decreased  by 7.4 percent to € 2.153 billion ( Q1 2023: € 2.005 billion), mainly'],

['negation', 'The Supply Chain Solutions segment benefited from increased  demand from pure e -commerce providers, general merchandise,  and food retailers, but order intake in the project business  remained subdued in the first three months impacted by customers’', 'The Supply Chain Solutions segment did not benefit  from increased demand from pure e - commerce providers, general merchandise, and food retailers, but order'],

['structure', 'This was mainly due to the continued stability of material purchase prices, increased productivity as a result of improved  material availability and revenue growth.', 'This was  to a small degree  due to the continued stability of material  purchase prices, increased productivity as a result of improved material'],

['worldknowledge', 'The Company  The KION Group is one of the world’s leading providers of industrial trucks and supply', 'The Company   The KION Group has removed industrial trucks from its product portfolio and focuses'],

['structure', 'the KION Group is the leading overseas manufacturer in China, and including domestic manufacturers, the third- largest supplier there.', 'the KION Group is the leading overseas manufacturer in China, and including domestic  manufacturers, the second -largest  supplier there.'],

['negation', 'The KION Group is also one of the world’s leading warehouse automation providers, based on 2022  revenue.', 'The KION Group has lost its leading position as a warehouse automation provider, based on 2022 revenue.'],

['entailment', 'At the end of 2023, more than 1.', 'At the end of 2023, more than 1.8 million industrial trucks of the KION Group were in  use by customers from a  small number  of sectors and of same  sizes on six continents.'],
]

In [ ]:
my_contradictions = [[premise, hypothesis, label] for [label, premise, hypothesis] in my_contradictions]

#### **Store results in persistent variable**

In [ ]:
langchain_sentences = my_contradictions
%store langchain_sentences

#### **Evaluation graph**

In [ ]:
# Data
categories = ['A', 'B', 'C', 'D', 'E']
values = [25, 16, 23, 13, 19]

# Creating the bar diagram
plt.figure(figsize=(6, 3))
plt.bar(categories, values, color='blue')
plt.title('Bar Diagram')
plt.xlabel('Labels')
plt.ylabel('Values')

# Show the plot
plt.show()

#### **Search for source document**

In [ ]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L12-v2")
check = Chroma(persist_directory="/home/user123/contradictory-information/src/vectorization/vector_store/kb_langchain", embedding_function=embedding_function)

In [ ]:
query = 'The adjusted EBIT margin of 7.9 percent came in  2.3 percentage points higher year -on-year, with significant improvements in both operating segments.'

check.similarity_search(query, k=50)

In [ ]:
['negation', 'The Supply Chain Solutions segment benefited from increased  demand from pure e -commerce providers, general merchandise,  and food retailers, but order intake in the project business  remained subdued in the first three months impacted by customers’', 'The Supply Chain Solutions segment did not benefit  from increased demand from pure e - commerce providers, general merchandise, and food retailers, but order']